In [2]:
import numpy as np
import random
import math
import random

In [3]:
def distance(p1, p2):
  """
  محاسبه فاصله اقلیدسی دو نقطه

  Args:
    p1: نقطه اول
    p2: نقطه دوم

  Returns:
    فاصله اقلیدسی بین دو نقطه
  """

  x1, y1 = p1
  x2, y2 = p2

  dx = x1 - x2
  dy = y1 - y2

  return math.sqrt(dx**2 + dy**2)

In [4]:
def calculate_distance(cities, solution):
    total_distance = 0
    for i in range(len(solution) - 1):
        city1 = cities[solution[i]]
        city2 = cities[solution[i + 1]]
        distance = ((city1[1] - city2[1])**2 + (city1[2] - city2[2])**2)**0.5
        total_distance += distance
    return total_distance


In [46]:
def randomGeneration():
  return np.random.permutation(100)

In [6]:
def read_city_data(filename):
    city_data = []
    with open(filename, 'r') as f:
        for line in f:
            city, lat, lon = line.strip().split(' ')
            city_data.append((city, int(lat), int(lon)))
            # id , lat (x), long(y) in tupple
    return city_data

In [7]:
city_data = read_city_data('city-cor.txt')
print(len(city_data))
# for city, lat, lon in city_data:
#     print(f"City: {city}, Latitude: {lat}, Longitude: {lon}")

100


In [47]:
print(city_data[0])

('1', 1380, 939)


In [48]:
def generate_random_chromosomes(population_size, num_cities=100):
    chromosomes = []
    for _ in range(population_size):
        chromosome = randomGeneration()
        # random.shuffle(chromosome)
        chromosomes.append(chromosome)
    return chromosomes

In [49]:
population = generate_random_chromosomes(500,100)
print(population[0])

[52 81 16 46 42 35 77 86 22 73 65 61 83  2 58 70 34 24 89 36 13 49 78 59
 47 75 93  3 74 71 50 12 80 15 41 82 88 26 19 18 72 84 55 90 56 23 27 30
  9 28 40 92 68  7 45 62 38  8 29  0 53 10 39 94 99  5 51 97 57 96 95 44
 20 69 32 11 79  4  1 25 85 67 21 87 31 91 98 37 66 48 76 43 60 14 17 33
 64 63 54  6]


In [50]:
def calculate_fitness(chromosome, cities=city_data):
    total_distance = 0
    if len(chromosome) != 100 :
      raise ValueError
    for i in range(100):
        city1_index = chromosome[i-1]
        city2_index = chromosome[i]
        
        city1 = cities[city1_index-1]
        city2 = cities[city2_index-1]

        latitude_diff = city1[1] - city2[1]
        longitude_diff = city1[2] - city2[2]

        distance = math.sqrt(latitude_diff**2 + longitude_diff**2)
        total_distance += distance
    return int(total_distance // 1)

In [51]:
calculate_fitness(population[0])

173972

In [52]:
def calculate_fitness_np(chromosome, cities=city_data):
    if len(chromosome) != 100 :
        print('wrong chromosome length')
        raise ValueError

    total_distance = 0
    for i in range(100):
        city1_index = chromosome[i-1]
        city2_index = chromosome[i]
        
        city1 = np.array(cities[city1_index-1][1:])
        city2 = np.array(cities[city2_index-1][1:])
        
        distance = np.linalg.norm(city1 - city2)
        total_distance += distance

    return int(total_distance // 1)

In [53]:
calculate_fitness_np(population[0])

173972

In [55]:
for chro in population :
  # print((chro[99]))
  fitforchro = calculate_fitness(chro)
print(fitforchro)

180790


In [16]:
# fitness_values = []
# for chromosome in population :
#     fitness_value = calculate_fitness(chromosome, cities = city_data)
#     fitness_values.append(fitness_value)
# fitness_values

In [57]:
fitness_values = np.array([calculate_fitness(chromosome, cities=city_data) for chromosome in population])
print(fitness_values)

[173972 174279 174317 162487 186247 174749 176827 165874 174307 176135
 178428 172275 165842 174263 175020 173609 170609 173392 172044 169869
 159394 170466 179766 176614 162260 168526 180162 174708 177367 181195
 167478 163123 179238 180461 159707 162715 180330 170140 165868 177303
 166091 155677 162782 173689 182379 179970 179670 173522 177026 176315
 165809 182241 169653 172977 174170 156679 170035 165559 162311 171345
 181656 168006 165572 157805 174367 172104 156512 165266 170838 162256
 185686 175626 164072 172917 175305 169036 162825 173155 162548 165939
 173784 175463 178082 151681 157944 168163 158819 183140 155419 171547
 150285 158873 165315 184209 176368 161637 174029 162905 167430 165720
 170156 162969 173470 167282 175166 174189 185009 167928 170108 168069
 177922 168675 185970 153267 177286 176305 176909 177655 175569 158652
 161223 166600 163969 169928 173859 171961 170256 181224 171832 163212
 160117 165086 177638 168110 165201 161584 182996 166616 176992 161094
 17320

In [18]:
def permutation_parent_selection(population, fitness_values, selection_method="roulette_wheel"):
  """Selects parents for a genetic algorithm using permutation representation.

  Args:
    population: A list of chromosomes in permutation representation.
    fitness_values: A list of fitness values for each chromosome in the population.
    selection_method: The parent selection method to use. Supported methods are
      "roulette_wheel" and "tournament".

  Returns:
    A tuple of two chromosomes selected as parents.
  """

  if selection_method == "roulette_wheel":
    return roulette_wheel_permutation_parent_selection(population, fitness_values)
  elif selection_method == "tournament":
    return tournament_permutation_parent_selection(population, fitness_values)
  else:
    raise ValueError("Unsupported parent selection method: {}".format(selection_method))


def roulette_wheel_permutation_parent_selection(population, fitness_values):
  """Selects parents using roulette wheel selection for permutation representation.

  Args:
    population: A list of chromosomes in permutation representation.
    fitness_values: A list of fitness values for each chromosome in the population.

  Returns:
    A tuple of two chromosomes selected as parents.
  """

  total_fitness = np.sum(fitness_values)
  fitness_probabilities = fitness_values / total_fitness
  cumulative_probabilities = np.cumsum(fitness_probabilities)

  random_value = random.random()
  parent1_index = 0
  for i, probability in enumerate(cumulative_probabilities):
    if random_value < probability:
      parent1 = population[i]
      break

  random_value = random.random()
  parent2_index = 0
  for i, probability in enumerate(cumulative_probabilities):
    if random_value < probability:
      parent2 = population[i]
      break

  return parent1, parent2


def tournament_permutation_parent_selection(population, fitness_values, tournament_size=2):
  """Selects parents using tournament selection for permutation representation.

  Args:
    population: A list of chromosomes in permutation representation.
    fitness_values: A list of fitness values for each chromosome in the population.
    tournament_size: The size of the tournament.

  Returns:
    A tuple of two chromosomes selected as parents.
  """

  parent1 = None
  parent2 = None

  for _ in range(tournament_size):
    random_index = random.randint(0, len(population) - 1)

    if parent1 is None or fitness_values[random_index] > fitness_values[parent1]:
      parent1 = population[random_index]

    if parent2 is None or fitness_values[random_index] > fitness_values[parent2]:
      parent2 = population[random_index]

  return parent1, parent2


In [58]:
roulette_wheel_permutation_parent_selection(population=population, fitness_values=fitness_values)

(array([46, 36, 97, 21, 64, 23, 50, 12,  4, 93, 42, 91, 56, 60, 52, 57, 73,
        17, 47, 31, 35, 32, 72,  8, 80, 67,  5,  0, 75, 15, 86, 95, 55, 96,
        82, 34, 70, 27,  3, 43, 18, 38, 88, 77,  7, 49, 19, 59, 11, 84, 71,
        62, 90, 92, 39, 44, 99, 53, 87, 20,  1, 14, 66, 65, 16, 30, 37, 63,
        29, 28, 85, 61, 24, 81, 74, 89, 83, 10,  9, 58, 78, 26, 41, 68, 94,
        79, 48, 54, 25, 98, 45, 69, 76, 33, 22, 13,  2, 51,  6, 40]),
 array([62, 76, 43, 46, 75, 98, 31, 95,  8, 81, 88, 80,  4, 87, 28,  3,  6,
        21, 69, 47, 55,  9, 13, 67, 15, 89, 97, 65, 24, 94, 84, 37, 41, 10,
        29, 92, 48, 40, 51, 72, 32, 19, 33, 91, 58,  5, 14, 61, 20, 78,  7,
        26, 57, 18, 27, 73, 60, 86, 44, 83, 71, 53, 82, 38, 36, 39, 11, 54,
        90, 59, 16, 77, 22, 68, 64, 79,  0, 45, 74, 25, 96,  1, 50, 17, 70,
        99, 23, 34, 12, 63, 56, 49, 35, 30, 42, 66, 85, 93,  2, 52]))

In [24]:
# import numpy as np

# def PMX_crossover(firstPermutation, secondPermutation):
#     length = len(firstPermutation)
    
#     first_Cross = np.random.randint(0, length // 2)
#     second_Cross = np.random.randint(length // 2 + 1, length) + 1

#     if first_Cross >= second_Cross:
#         # swap the crossover points if necessary
#         first_Cross, second_Cross = second_Cross, first_Cross
    
#     segment1 = firstPermutation[first_Cross:second_Cross]
#     segment2 = secondPermutation[first_Cross:second_Cross]
#     mapping = np.zeros(length, dtype=int)
    
#     for i in range(len(segment1)):
#         mapping[segment1[i]] = segment2[i]
#         mapping[segment2[i]] = segment1[i]
#     if np.unique(mapping).size != length:
#         # raise an error if the mapping array is invalid
#         raise ValueError("Invalid mapping array")
#     child1 = np.zeros(length, dtype=int)
#     child2 = np.zeros(length, dtype=int)
    
#     for i in range(length):
#         if i >= first_Cross and i <= second_Cross:
#             child1[i] = segment2[i - first_Cross]
#             child2[i] = segment1[i - first_Cross]
#         else:
#             if mapping[firstPermutation[i]] == 0:
#                 child1[i] = firstPermutation[i]
#             else:
#                 child1[i] = mapping[firstPermutation[i]]
            
#             if mapping[secondPermutation[i]] == 0:
#                 child2[i] = secondPermutation[i]
#             else:
#                 child2[i] = mapping[secondPermutation[i]]
    
#     return child1, child2


In [59]:
import numpy as np

def PMX_crossover(firstPermutation, secondPermutation):
    length = len(firstPermutation)
    
    first_Cross = np.random.randint(0, length // 2)
    second_Cross = np.random.randint(length // 2 + 1, length) + 1

    if first_Cross >= second_Cross:
        # swap the crossover points if necessary
        first_Cross, second_Cross = second_Cross, first_Cross
    
    segment1 = firstPermutation[first_Cross:second_Cross]
    segment2 = secondPermutation[first_Cross:second_Cross]
    mapping = np.zeros(length, dtype=int)
    
    for i in range(len(segment1)):
        mapping[segment1[i]] = segment2[i]
        # print(segment2[i])
        # print(mapping.length)
        mapping[segment2[i]] = segment1[i]
    # if np.unique(mapping).size != length:
    #     # raise an error if the mapping array is invalid
    #     raise ValueError("Invalid mapping array")
    child1 = np.zeros(length, dtype=int)
    child2 = np.zeros(length, dtype=int)
    
    for i in range(length):
        if i >= first_Cross and i <= second_Cross:
            child1[i] = segment2[i - first_Cross - 1 ] # use i - first_Cross here
            child2[i] = segment1[i - first_Cross - 1] # use i - first_Cross here
        else:
            if mapping[firstPermutation[i]] == 0:
                child1[i] = firstPermutation[i]
            else:
                child1[i] = mapping[firstPermutation[i]]
            
            if mapping[secondPermutation[i]] == 0:
                child2[i] = secondPermutation[i]
            else:
                child2[i] = mapping[secondPermutation[i]]
    
    return child1, child2


In [60]:
offspring1, offspring2 = PMX_crossover(population[0], population[1])
print("Offspring 1:", offspring1)
print("Offspring 2:", offspring2)

Offspring 1: [52 93 16 90 42 35 39 86 12 73 15 61 72  2 30 70 34 45 49 13 14 36 89 18
  0 45 27 81 91 67 12 76 22 51 65 17 53 26 19  3 39 83 37 72 46 15 97 78
 58 57 79 94 41  9 64 24 88 38 21 98  5 50 71 77 14  5 80 23  9 96 95 44
 20 69 32 11 28  4  1 25 85 74  8 87 31  3 29 84 66 48 50 43 60 94 41 33
  7 63 54  6]
Offspring 2: [73 72 27 85  4 71  9 88 67 69 61 60 37 51 25  1 44 94 45 53 94 13 49 78
 59 47 75 93  3 74 71 50 12 80 15 41 82 88 26 19 18 72 84 55 90 56 23 27
 30  9 28 40 92 68  7 45 62 38  8 29  0 53 10 39 94 81 11 15 35 66 32 59
  6 79 36 63 16 96 87 46 34 52 43 21 89 41  2 70 64 33 86 95 31 48 99 98
 20 42 54 97]


In [61]:
firstPermutation = np.random.permutation(100) # generate a random permutation of integers from 0 to 99
print(firstPermutation.shape)
secondPermutation = np.random.permutation(100) # generate another random permutation of integers from 0 to 99
PMX_crossover(firstPermutation, secondPermutation) # apply the PMX

(100,)


(array([ 3, 85, 13, 42, 36,  3, 19, 57, 11, 32, 53,  5, 45, 28,  4, 39, 69,
        52, 72, 66, 29, 77, 80,  8, 40, 44, 13, 70, 25, 95, 93, 96, 98, 41,
        56, 12, 46, 30, 58,  9, 74, 62,  2, 89, 22,  1, 83, 82, 51, 23, 78,
        16, 81, 99, 64, 15, 42, 61, 97, 64, 60, 48, 90, 24, 50, 33, 35, 92,
         6, 42, 18,  0, 10, 15, 73, 44, 17,  8, 41, 51, 68, 59,  4, 34, 54,
        55, 78, 87, 37, 62, 89, 32, 67, 21, 75, 84, 38, 58, 91, 63]),
 array([ 3, 48, 97,  7, 73, 49,  8, 32, 58, 88, 15, 17, 21, 67,  1, 30,  3,
        78, 89, 50, 94, 62, 27, 65, 41, 14, 43, 68, 20, 42, 13, 92, 44, 80,
        54, 35, 37, 31, 26, 84, 51, 29, 81, 82,  4, 25, 55, 39, 47, 64, 79,
        61, 71, 60, 86,  2,  7,  6, 51,  9, 10, 28, 32, 38, 30, 24,  0, 76,
        75, 46, 12, 85, 90, 81,  8, 59, 66, 16, 13, 42, 83, 99, 45, 34, 29,
         5, 44, 63, 18, 33, 87, 36, 70, 64, 96, 91, 78, 58, 80, 25]))

In [62]:
def cxPartialyMatched(ind1, ind2):
    size = min(len(ind1), len(ind2))
    p1, p2 = [0] * size, [0] * size

    # Initialize the position of each indices in the individuals
    for i in range(size):
        p1[ind1[i]] = i
        p2[ind2[i]] = i
    # Choose crossover points
    cxpoint1 = random.randint(0, size)
    cxpoint2 = random.randint(0, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:  # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    # Apply crossover between cx points
    for i in range(cxpoint1, cxpoint2):
        # Keep track of the selected values
        temp1 = ind1[i]
        temp2 = ind2[i]
        # Swap the matched value
        ind1[i], ind1[p1[temp2]] = temp2, temp1
        ind2[i], ind2[p2[temp1]] = temp1, temp2
        # Position bookkeeping
        p1[temp1], p1[temp2] = p1[temp2], p1[temp1]
        p2[temp1], p2[temp2] = p2[temp2], p2[temp1]

    return ind1, ind2
offspring1, offspring2 = cxPartialyMatched(population[0], population[1])
print("Offspring 1:", offspring1)
print("Offspring 2:", offspring2)

Offspring 1: [52 81 16 90 42 35 77 86 12 73 15 61 72  2 30 70 34 45 89 36 13 49 27 59
 47 75 93 88 74 71 76 22 51 56 92 53 62 19  3 39 55 37 83 46 65 97 78 58
 68 79 94 17 57 64 24 26 38 21 98  0 82 10 18 40 99  5 80 23  9 96 95 44
 20 69 32 11 28  4  1 25 85 67  8 87 31 91 29 84 66 48 50 43 60 14 41 33
  7 63 54  6]
Offspring 2: [73 83 75 85  4 10 57 26 74 69 61 60 37 51 25  1 44 94 47 53 36 89 39  0
 24 78 81 91 67 22 50 12 80 15 41 82 19  3 88 18 72 84 55 90 56 23 27 30
  9 28 40 92 68  7 45 62 38  8 29  5 76 71 77 14 58 93 11 65 35 66 32 59
  6 79 13 63 16 96 87 46 34 52 43 21 49 17  2 70 64 33 86 95 31 48 99 98
 20 42 54 97]


In [63]:
def PMX_crossover(parent1, parent2, seed):
    '''
    parent1 and parent2 are 1D np.array
    '''
    rng = np.random.default_rng(seed=seed)

    cutoff_1, cutoff_2 = np.sort(rng.choice(np.arange(len(parent1)+1), size=2, replace=False))

    def PMX_one_offspring(p1, p2):
        offspring = np.zeros(len(p1))

        # Copy the mapping section (middle) from parent1
        offspring[cutoff_1:cutoff_2] = p1[cutoff_1:cutoff_2]

        # copy the rest from parent2 (provided it's not already there
        for i in np.concatenate([np.arange(0,cutoff_1), np.arange(cutoff_2,len(p1))]):
            candidate = p2[i]
            while candidate in p1[cutoff_1:cutoff_2]: # allows for several successive mappings
                print(f"Candidate {candidate} not valid in position {i}") # DEBUGONLY
                candidate = p2[np.where(p1 == candidate)[0]]
            offspring[i] = candidate
        return offspring

    offspring1 = PMX_one_offspring(parent1, parent2)
    offspring2 = PMX_one_offspring(parent2, parent1)

    return offspring1, offspring2
offspring1, offspring2 = PMX_crossover(population[0], population[1],23)
print("Offspring 1:", offspring1)
print("Offspring 2:", offspring2)

Candidate 73 not valid in position 0
Candidate 83 not valid in position 1
Candidate [55] not valid in position 1
Candidate [72] not valid in position 1
Candidate [37] not valid in position 1
Candidate 75 not valid in position 2
Candidate [78] not valid in position 2
Candidate [27] not valid in position 2
Candidate [39] not valid in position 2
Candidate [18] not valid in position 2
Candidate [77] not valid in position 2
Candidate [57] not valid in position 2
Candidate [68] not valid in position 2
Candidate [9] not valid in position 2
Candidate [35] not valid in position 2
Candidate [10] not valid in position 2
Candidate [71] not valid in position 2
Candidate [22] not valid in position 2
Candidate [12] not valid in position 2
Candidate [74] not valid in position 2
Candidate 59 not valid in position 71
Candidate [0] not valid in position 71
Candidate [5] not valid in position 71
Candidate [93] not valid in position 71
Candidate 79 not valid in position 73
Candidate 13 not valid in positio

C:\Users\blues\AppData\Local\Temp\ipykernel_21740\1868980976.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  offspring[i] = candidate


In [64]:
import numpy as np

def pmx(parent1, parent2):
    size = len(parent1)
    # Choose crossover points
    idx1, idx2 = np.sort(np.random.choice(range(size), 2, replace=False))

    # Initialize offspring
    offspring1, offspring2 = np.zeros(size, dtype=int), np.zeros(size, dtype=int)

    # Copy segments from parents
    offspring1[idx1:idx2+1], offspring2[idx1:idx2+1] = parent2[idx1:idx2+1], parent1[idx1:idx2+1]

    # Fill the remaining positions
    for i in list(range(idx1)) + list(range(idx2+1, size)):
        offspring1[i] = parent1[i] if parent1[i] not in offspring1 else parent2[np.where(parent1 == parent1[i])[0][0]]
        offspring2[i] = parent2[i] if parent2[i] not in offspring2 else parent1[np.where(parent2 == parent2[i])[0][0]]

    return offspring1, offspring2

# Test
parent1 = np.array([1, 2, 3, 4, 5, 6, 7, 8])
parent2 = np.array([2, 4, 6, 8, 7, 5, 3, 1])

offspring1, offspring2 = pmx(parent1, parent2)
print("Offspring 1:", offspring1)
print("Offspring 2:", offspring2)


Offspring 1: [2 4 6 8 7 5 3 1]
Offspring 2: [1 2 3 4 5 6 7 8]


In [65]:
def pmx_crossover(parent1, parent2):
    """Performs Partially Matched Crossover (PMX) recombination on two chromosomes.

    Args:
        parent1: The first parent chromosome.
        parent2: The second parent chromosome.

    Returns:
        A new chromosome resulting from the crossover.
    """

    mapping = {}
    crossover_points = random.sample(range(1, len(parent1)), 2)

    for i in range(crossover_points[0], crossover_points[1]):
        gene1 = parent1[i]
        gene2 = parent2[i]

        mapping[gene1] = gene2
        mapping[gene2] = gene1

    offspring = list(parent1)

    for i in range(len(offspring)):
        if i in mapping:
            offspring[i] = mapping[offspring[i]]

    return offspring

In [23]:
def erx_crossover(parent1, parent2):
    """Performs Edge Recombination Crossover (ERX) on two chromosomes.

    Args:
        parent1: The first parent chromosome.
        parent2: The second parent chromosome.

    Returns:
        A new chromosome resulting from the crossover.
    """

    offspring = []
    visited = set()

    # Start from a random gene in parent1
    gene = random.choice(parent1)
    offspring.append(gene)
    visited.add(gene)

    while len(offspring) < len(parent1):
        # Find the next gene to add
        next_gene = None

        # Check if the next gene in parent2 is not in the visited set
        next_gene_in_parent2 = parent2[(parent1.index(gene) + 1) % len(parent1)]
        if next_gene_in_parent2 not in visited:
            next_gene = next_gene_in_parent2

        # Check if the next gene in parent1 is not in the visited set
        if next_gene is None:
            next_gene_in_parent1 = parent1[(parent2.index(gene) + 1) % len(parent2)]
            if next_gene_in_parent1 not in visited:
                next_gene = next_gene_in_parent1

        # If no valid next gene found, break
        if next_gene is None:
            break

        # Add the next gene to the offspring and mark it as visited
        offspring.append(next_gene)
        visited.add(next_gene)

        # Update the current gene
        gene = next_gene

    return offspring

In [24]:
def ox_crossover(parent1, parent2):
    """Performs Order Crossover (OX) on two chromosomes.

    Args:
        parent1: The first parent chromosome.
        parent2: The second parent chromosome.

    Returns:
        A new chromosome resulting from the crossover.
    """

    crossover_points = random.sample(range(1, len(parent1)), 2)
    slice_start, slice_end = crossover_points[0], crossover_points[1]

    slice1 = parent1[slice_start:slice_end]
    remaining_genes1 = set(parent1) - set(slice1)

    mapping = {}
    for i in range(slice_start, slice_end):
        gene1 = parent1[i]
        gene2 = parent2[i]

        if gene1 in remaining_genes1:
            mapping[gene1] = gene2
            remaining_genes1.remove(gene1)

    offspring = []
    for gene in parent2:
        if gene in mapping:
            offspring.append(mapping[gene])
        elif gene not in slice1:
            offspring.append(gene)

    for gene in slice1:
        offspring.append(gene)

    return offspring

In [25]:
def cycle_crossover(parent1, parent2):
    """
    Performs Cycle Crossover (CX) on two chromosomes.

    Args:
        parent1: The first parent chromosome.
        parent2: The second parent chromosome.

    Returns:
        A new chromosome resulting from the crossover.
    """

    cycle_start = random.randrange(len(parent1))
    cycle_genes1 = set(parent1[cycle_start:])
    cycle_genes2 = set(parent2[cycle_start:])

    offspring = list(parent1[:cycle_start])
    for gene in parent2:
        if gene in cycle_genes1:
            offspring.append(gene)
            cycle_genes1.remove(gene)

            if not cycle_genes1:
                break
        else:
            cycle_genes2.add(gene)

    for gene in cycle_genes2:
        offspring.append(gene)

    return offspring

In [26]:
def swap_mutation(chromosome):
    """Performs Swap mutation on a chromosome.

    Args:
        chromosome: The chromosome to mutate.

    Returns:
        A mutated chromosome.
    """

    i, j = random.sample(range(len(chromosome)), 2)
    chromosome[i], chromosome[j] = chromosome[j], chromosome[i]
    return chromosome

In [27]:
def scramble_mutation(chromosome):
    """Performs Scramble mutation on a chromosome.

    Args:
        chromosome: The chromosome to mutate.

    Returns:
        A mutated chromosome.
    """

    mutation_length = random.randrange(2, len(chromosome))
    start_index = random.randrange(len(chromosome) - mutation_length)

    sub_chromosome = chromosome[start_index:start_index + mutation_length]
    random.shuffle(sub_chromosome)
    chromosome[start_index:start_index + mutation_length] = sub_chromosome

    return chromosome

In [28]:
def insert_mutation(chromosome):
    """Performs Insert mutation on a chromosome.

    Args:
        chromosome: The chromosome to mutate.

    Returns:
        A mutated chromosome.
    """

    i = random.randrange(len(chromosome))
    j = random.randrange(len(chromosome))

    if i != j:
        gene = chromosome.pop(i)
        chromosome.insert(j, gene)

    return chromosome

In [29]:
def inversion_mutation(chromosome):
    """Performs Inversion mutation on a chromosome.

    Args:
        chromosome: The chromosome to mutate.

    Returns:
        A mutated chromosome.
    """

    mutation_length = random.randrange(2, len(chromosome))
    start_index = random.randrange(len(chromosome) - mutation_length)

    sub_chromosome = chromosome[start_index:start_index + mutation_length]
    sub_chromosome.reverse()
    chromosome[start_index:start_index + mutation_length] = sub_chromosome

    return chromosome

In [51]:
# def roulette_wheel_selection(population, fitness_values):
#     total_fitness = np.sum(fitness_values)
#     fitness_probabilities = fitness_values / total_fitness
#     cumulative_probabilities = np.cumsum(fitness_probabilities)

#     random_index = random.random()
#     for i, probability in enumerate(cumulative_probabilities):
#         # print(f'i is {i}')
#         if random_index < probability:
#             return population[i]
def roulette_wheel_selection(population, fitness_values):
    total_fitness = np.sum(fitness_values)
    fitness_probabilities = fitness_values / total_fitness
    cumulative_probabilities = np.cumsum(fitness_probabilities)

    random_value = random.random()
    for i, probability in enumerate(cumulative_probabilities):
        if random_value < probability:
            selected_chromosome = population[i]
            break

    # Shuffle the selected chromosome to ensure no bias in selecting the next parent
    # np.random.shuffle(selected_chromosome)

    return selected_chromosome

def tournament_selection(population , cities = city_data, tournament_size = 5):
    # Create a tournament
    tournament = random.sample(population, tournament_size)
    # Select the best individual among the tournament
    best_individual = min(tournament, key=lambda x: calculate_distance(cities, x))
    return best_individual

def genetic_algorithm(population_size=500, crossover_rate=0.7, mutation_rate=0.1, max_generations=700,cities = city_data):
    chromosomes = generate_random_chromosomes(500,100)
    fitness_values = []
    for chromosome in chromosomes :
        fitness_value = calculate_fitness_np(chromosome, cities)
        fitness_values.append(fitness_value)

    generation_without_improvement = 0

    for generation in range(max_generations):
        # Select parents using roulette wheel selection
        parents = []
        for i in range(population_size):
            parent1 , parent2 = roulette_wheel_permutation_parent_selection(chromosomes, fitness_values)
            # parent2 = roulette_wheel_permutation_parent_selection(chromosomes, fitness_values)

            parents.append((parent1, parent2))

        # Apply crossover
        # offspring = []
        # for parent1, parent2 in parents:
        #     offspring1, offspring2 = pmx_crossover(chromosomes[parent1], chromosomes[parent2])
        #     offspring.append(offspring1)
        #     offspring.append(offspring2)
        # Apply crossover with specified rate
        offspring = []
        for parent1, parent2 in parents:
            print(parent1)
            # break
            if random.random() < crossover_rate:
                offspring1 = pmx(parent1, parent2)
                offspring2 = pmx(parent1, parent2)
            else:
                offspring1 = parent1
                offspring2 = parent2

            offspring.append(offspring1)
            offspring.append(offspring2)

        # Apply mutation
        for offspring in offspring:
            if random.random() < mutation_rate:
                offspring = swap_mutation(offspring)

        # Evaluate offspring fitness
        offspring_fitness_values = calculate_fitness(offspring, cities)

        # Combine parents and offspring to form a new population
        new_population = np.empty_like(chromosomes)
        new_fitness_values = np.empty_like(fitness_values)

        for i in range(population_size):
            if offspring_fitness_values[i] > fitness_values[i]:
                new_population[i] = offspring[i]
                new_fitness_values[i] = offspring_fitness_values[i]
            else:
                new_population[i] = chromosomes[i]
                new_fitness_values[i] = fitness_values[i]

        chromosomes = new_population
        fitness_values = new_fitness_values

        # Check for stopping condition
        if np.array_equal(fitness_values, offspring_fitness_values):
            generation_without_improvement += 1

            if generation_without_improvement >= 20:
                break
        else:
            generation_without_improvement = 0

    best_chromosome = chromosomes[fitness_values.argmax()]
    best_fitness = np.max(fitness_values)

    return best_chromosome, best_fitness


In [52]:
result = genetic_algorithm()

[54, 91, 74, 82, 98, 18, 87, 17, 15, 32, 39, 33, 28, 47, 85, 53, 30, 75, 84, 31, 45, 9, 26, 88, 36, 6, 96, 59, 94, 37, 48, 13, 89, 42, 72, 24, 57, 34, 93, 29, 77, 41, 35, 83, 80, 79, 61, 69, 68, 92, 19, 86, 51, 20, 23, 8, 50, 12, 4, 1, 76, 81, 44, 43, 49, 22, 2, 70, 7, 100, 64, 25, 62, 27, 95, 52, 78, 11, 38, 16, 71, 90, 67, 3, 60, 58, 66, 40, 10, 14, 63, 73, 56, 46, 5, 97, 21, 99, 55, 65]
[62, 76, 72, 38, 40, 100, 94, 53, 13, 24, 30, 34, 41, 49, 54, 58, 46, 10, 67, 80, 2, 1, 18, 88, 25, 70, 98, 9, 66, 3, 57, 29, 22, 7, 23, 32, 8, 73, 21, 90, 97, 85, 60, 87, 84, 83, 27, 95, 5, 35, 42, 96, 31, 99, 14, 17, 82, 78, 69, 26, 75, 37, 68, 59, 77, 79, 6, 81, 16, 89, 48, 45, 44, 33, 19, 4, 64, 93, 51, 20, 74, 61, 43, 52, 28, 71, 55, 63, 11, 86, 56, 47, 50, 92, 91, 12, 39, 15, 65, 36]


C:\Users\blues\AppData\Local\Temp\ipykernel_36744\557155559.py:16: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  offspring1[i] = parent1[i] if parent1[i] not in offspring1 else parent2[np.where(parent1 == parent1[i])[0][0]]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
def calculate_distance(cities, solution):
    total_distance = 0
    for i in range(len(solution) - 1):
        city1 = cities[solution[i]]
        city2 = cities[solution[i + 1]]
        distance = ((city1[1] - city2[1])**2 + (city1[2] - city2[2])**2)**0.5
        total_distance += distance
    return total_distance

def generate_individual(cities):
    return random.sample(range(len(cities)), len(cities))

def generate_population(cities, size):
    return [generate_individual(cities) for _ in range(size)]

def crossover(parent1, parent2):
    child = parent1[:len(parent1)//2]
    child += [gene for gene in parent2 if gene not in child]
    return child

def mutate(individual):
    index1 = random.randint(0, len(individual) - 1)
    index2 = random.randint(0, len(individual) - 1)
    individual[index1], individual[index2] = individual[index2], individual[index1]

def genetic_algorithm(cities, pop_size, generations):
    population = generate_population(cities, pop_size)
    for _ in range(generations):
        population.sort(key=lambda x: calculate_distance(cities, x))
        if calculate_distance(cities, population[0]) == calculate_distance(cities, population[1]):
            break
        next_gen = population[:pop_size//2]
        for i in range(pop_size//2, pop_size, 2):
            parent1 = random.choice(next_gen)
            parent2 = random.choice(next_gen)
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)
            if random.random() < 0.1:
                mutate(child1)
            if random.random() < 0.1:
                mutate(child2)
            next_gen += [child1, child2]
        population = next_gen
    return population[0]
